## Read the csv file, drop the duplicate(based on converstaionID) and remove unncessary column

In [1]:
import pandas as pd
import os
from utils import show_df

os.chdir(r'D:\Project\Twitter_depression_detector\data')
os.getcwd()


'D:\\Project\\Twitter_depression_detector\\data'

In [2]:
# Reads the json generated from the CLI commands above and creates a pandas dataframe
tweets_df = pd.read_csv('Depression_merged.csv')
tweets_df=tweets_df.drop_duplicates(subset=['conversationId'])
tweets_df=tweets_df.drop(columns=['Unnamed: 0'],axis=1)


## Search for all the hastags in tweet using regex

In [3]:
tweets_df['hashtags']=tweets_df.content.str.findall(r'#.*?(?=\s|$)')
print(tweets_df['hashtags'])

show_df(tweets_df)

0                                                        []
1                                                        []
2                                                        []
3                                                        []
4                                                        []
                                ...                        
140828                                                   []
140829                                                   []
140830                                              [#gawx]
140831    [#essentialoils, #aromatherapy, #essentialoil,...
140832                                                   []
Name: hashtags, Length: 125128, dtype: object


Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [], 'inp…

## get Count the hashtags and remove the hastags related to medical terms

In [4]:
tweets_df.hashtags.value_counts().head(20)
medical_terms = ["#mentalhealth", "#health", "#happiness", "#mentalillness", "#happy", "#joy", "#wellbeing"]
mask1 = tweets_df.hashtags.apply(lambda x: any(item for item in medical_terms if item in x))
print(tweets_df[mask1].content.tail())
tweets_df[mask1==False].content.head(10)
tweets_df=tweets_df[mask1==False]

#show_df(tweets_df)


140660    Untreated depression can have tragic consequen...
140719    New blog online #mentalhealth #anxiety #bpd #d...
140782    Got 60 seconds? Take a #mentalhealth minute to...
140785    Have you seen our Foundry webpage? 👉 https://t...
140826    I love this quote! 💜\n\nRemember to always #Be...
Name: content, dtype: object


## remove tweets which contains too many hastags

In [5]:
mask2 = tweets_df.hashtags.apply(lambda x: len(x) < 4)
tweets_df=tweets_df[mask2]
tweets_df.hashtags.value_counts().head(20)

#show_df(tweets_df)


TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                                                                  105416
[#depression]                                                          627
[#Depression]                                                          120
[#onlinetherapy, #anxiety, #depression]                                 67
[#CelebrateHoli2020InVedicWay]                                          62
[#VoiceOfDepressionKE]                                                  58
[#Anxiety, #Depression]                                                 56
[#gawx]                                                                 56
[#coronavirus]                                                          51
[#Anxiety, #Depression.]                                                48
[#depression, #anxiety]                                                 43
[#MentalHealthMatters]                                                  42
[#Anxiety, #Depression, #Addiction]                                     41
[#anxiety, #depression]  

## remove tweets with @ mentions as they are sometimes retweets


In [6]:
import ast
print("Len of dataset: ",len(tweets_df))
# the mentioned user were stored as string so converted them to list
tweets_df['mentionedUsers']=[ast.literal_eval(mentioneduser) if type(mentioneduser)!=float else str(mentioneduser) for mentioneduser in tweets_df['mentionedUsers']]
mask3 = tweets_df.mentionedUsers.apply(lambda x: len(x) < 5)
tweets_df = tweets_df[mask3]
print("Len of dataset: ",len(tweets_df))

Len of dataset:  116846
Len of dataset:  115785


In [7]:
#show_df(tweets_df)


## Remove tweets containing URLS as they might be promotional msgs

In [8]:
import ast
print(len(tweets_df))
tweets_df['outlinks']=[ast.literal_eval(outlink) for outlink in tweets_df['outlinks']]
mask4 = tweets_df.outlinks.apply(lambda x: len(x)==0)
tweets_df = tweets_df[mask4]
print(len(tweets_df))

115785
95842


In [9]:
#show_df(tweets_df)

## Feature engineering, a column featuring count of the mentioneduser

In [10]:
tweets_df['mentionedUserCount']=[len(mentionedUser) if type(mentionedUser)==list else 0 for mentionedUser in tweets_df['mentionedUsers']]

In [11]:
#show_df(tweets_df)

In [12]:
#tweets_df.to_csv('cleaned_tweets.csv')

## see the difference between two columns content and renderedcontent

In [13]:
count=0
for i in range(len(tweets_df)):
    if tweets_df.iloc[i]['renderedContent']==tweets_df.iloc[i]['content']:
        count+=1
print(count)        

95842


## select the useful columns


In [14]:
tweets_df.columns

Index(['Unnamed: 0.1', 'url', 'date', 'content', 'renderedContent', 'id',
       'user', 'outlinks', 'tcooutlinks', 'replyCount', 'retweetCount',
       'likeCount', 'quoteCount', 'conversationId', 'lang', 'source',
       'sourceUrl', 'sourceLabel', 'media', 'retweetedTweet', 'quotedTweet',
       'mentionedUsers', 'hashtags', 'mentionedUserCount'],
      dtype='object')

In [15]:
#tweets_cleaned_df=tweets_df[['url','date','user','renderedContent','replyCount','retweetCount','likeCount','quoteCount','mentionedUserCount']]
tweets_cleaned_df=tweets_df[['renderedContent','mentionedUserCount']]

In [16]:
#show_df(tweets_cleaned_df)

## Exporting the cleaned tweets

In [18]:
tweets_cleaned_df.to_csv('cleaned_merged_depression_tweets.csv',index=False)